In [17]:
from glob import glob
import os
import shutil
import random
from tqdm import tqdm
import json
import cv2
from threading import Thread

In [18]:
labels1 = glob('../../datasets/aihub_safety/train/*/*/*/*.json')
labels2 = glob('../../datasets/aihub_safety/train/*/*/*.json')
labels = labels1 + labels2

images = glob('../../datasets/aihub_safety/train/*/*.jpg')



In [19]:
a = set()

for i in labels:
    a.add(os.path.dirname(i))
a = list(a)

In [20]:
b = set()
for i in labels:
    b.add(os.path.basename(i))


In [21]:
len(images) # 이미지 데이터 수

1862288

In [22]:
len(b) # 라벨 데이터 수 

1926980

In [23]:
# image 데이터 수에 비해 라벨 데이터 수가 더 많음 
# 데이터 수 자체가 많기 때문에 이미지 데이터와 매핑 되지 않은 라벨 데이터를 삭제하여 사용

In [24]:
# 좌표 순서 바뀐 오류 수정 
def coord_organize(box):
    return [min(box[0],box[2]), min(box[1],box[3]), max(box[0],box[2]), max(box[1],box[3])]

def cvt_yoloformat(xyxy,w,h):
    box_w = xyxy[2] - xyxy[0]
    box_h = xyxy[3] - xyxy[1]
    return [(xyxy[0] + box_w/2)/w ,(xyxy[1] + box_h/2)/h, box_w/w, box_h/h ]
    

In [25]:
def read_json(path,img_f):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
            annos = data['annotations']
    except Exception as e:
        try:
            with open(path, 'r',encoding='utf-8-sig') as f:
                data = json.load(f)
                annos = data['annotations']
        except Exception as e:
            print(e, '\n json data 읽기 실패')

    
    data_extst = False
    new_datas = []

    # w, h 는 json 파일에서의 주어진 w,h 를 쓰면 잘못된 경우 발견되어 이미지를 읽어서 구함.
    img = cv2.imread(img_f[0])
    w,h = img.shape[1],img.shape[0]

    if len(annos):
        for anno in annos:
            try:
                cls = int(anno['class'] )
                box = anno['box']
            except:
                cls = None
                box = []
            # print('box : ', box, 'cls : ', cls)
            # print((len(box)) and (int(cls) < 20))
            if (len(box)) and (int(cls) < 20):
                data_extst = True

                box = coord_organize(box) #
                box = cvt_yoloformat(box,w,h)
                new_data = str(int(cls) - 1) + ' ' + ' '.join(str(k) for k in box)  + '\n'

                new_datas.append(new_data)
    
    return  data_extst,new_datas
    
def check_name(dirnum,path):

    list01 = ['1.안전보호구만','2.중장비만','3.구조물_비계만','4.자재및공구만','5.전체']

    count = 0
    only = 0

    bool_list = [False,False,False,False,False]

    img_f = glob('../../datasets/aihub_safety/train/[[]원천]' + j.split('[라벨]')[1][0] + '*/' + os.path.basename(j)[:-4] +'jpg')

    
    if len(img_f) == 0:
        return 0

    for i in range(5):
        if (int(dirnum)-1) == i:
            bool_list[int(dirnum)-1] = True
            continue        

        checkname = os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[i]}/' + os.path.basename(path)

        if os.path.exists(checkname):
            bool_list[i] = True
    

    if not (bool_list[0] or bool_list[1]):
        if bool_list[4]:
            data_extst, _= read_json(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[4]}/' + os.path.basename(path),img_f)
            if data_extst:
                return [4,img_f]
            else:
                return 0
        else:
            return 0
    elif bool_list[0] and bool_list[1]:
        if bool_list[4]:
            return [5,img_f] 
        else:
            return [3,img_f] 
    else:
        if bool_list[4]:
            return [5,img_f] 
        elif bool_list[0]:
            return [1,img_f] 
        elif bool_list[1]:
            return [2,img_f] 

In [26]:
def make_txtfile(des_path ,dirnum,ret,path):
    list01 = ['1.안전보호구만','2.중장비만','3.구조물_비계만','4.자재및공구만','5.전체']
    jsonfiles = []
    if ret[0] == 1:
        jsonfiles.append(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[0]}/' + os.path.basename(path))
    elif ret[0] == 2:
        jsonfiles.append(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[1]}/' + os.path.basename(path))
    elif ret[0] == 3:
        jsonfiles.append(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[0]}/' + os.path.basename(path))
        jsonfiles.append(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[1]}/' + os.path.basename(path))
    elif ret[0] in [4,5]:
        jsonfiles.append(os.path.dirname(path).split(f'/{dirnum}.')[0] + f'/{list01[4]}/' + os.path.basename(path))


    img_f = ret[1]  
    datas = []
    # read jsonfiles
    for i in jsonfiles:
        data_extst,new_datas = read_json(i,img_f)
        if data_extst:
            datas += new_datas

    if not os.path.exists(des_path + '/images'):
        os.mkdir(des_path + '/images')
    if not os.path.exists(des_path + '/labels'):
        os.mkdir(des_path + '/labels')

    # write txtfiles
    if datas: # data exist
        # count += 1
        
        try:
            
            shutil.copy(img_f[0] , des_path + '/images') # image copy
            with open(des_path + '/labels/' + os.path.basename(j)[:-4] + 'txt', 'w') as f: # write converted labeled file
                for p in new_datas:
                    f.write(p)
        except:
            print('failed to write files or can\'t find image file', img_f)
    




def read_json(path,img_f):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
            annos = data['annotations']
    except Exception as e:
        try:
            with open(path, 'r',encoding='utf-8-sig') as f:
                data = json.load(f)
                annos = data['annotations']
        except Exception as e:
            print(e, '\n json data 읽기 실패')

    
    data_extst = False
    new_datas = []

    # w, h 는 json 파일에서의 주어진 w,h 를 쓰면 잘못된 경우 발견되어 이미지를 읽어서 구함.
    try:
        img = cv2.imread(img_f[0])
        w,h = img.shape[1],img.shape[0]
    except:
        return False,[]

    if len(annos):
        for anno in annos:
            try:
                cls = int(anno['class'] )
                box = anno['box']
            except:
                cls = None
                box = []

            if (len(box)) and (int(cls) < 20):
                data_extst = True

                box = coord_organize(box) #
                box = cvt_yoloformat(box,w,h)
                new_data = str(int(cls) - 1) + ' ' + ' '.join(str(k) for k in box)  + '\n'

                new_datas.append(new_data)
    
    return  data_extst,new_datas




In [29]:
# def data_classify(labels,results,i ):
counts = {'0' : 0, '1' : 0 , '2' : 0, '3' : 0, '4' : 0 , '5' : 0}
for i,j in tqdm(enumerate(labels)):
    dirnum = os.path.dirname(j).split('/')[-1][0]
    filenum = os.path.basename(j)

    
    ret = check_name(dirnum, j)

    if isinstance(ret, int):
        counts['0'] +=1
        continue
    else:
        if ret[0] ==1:
            counts['1'] +=1
        elif ret[0] ==2:
            counts['2'] +=1
        elif ret[0] ==3:
            counts['3'] +=1
        elif ret[0] ==4:
            counts['4'] +=1
        elif ret[0] ==5:
            counts['5'] +=1


    des_path = '../../datasets/aihub_safety/human_car_ver2'
    make_txtfile(des_path,dirnum,ret,j)        

    
    # results[i] = counts



8609it [01:07, 101.38it/s] 

In [28]:
len(labels)

2304631

In [112]:
# start_idx = 0
# end_idx = 2304631
# indice = list(range(start_idx,end_idx,end_idx//20))
# indice[-1] = end_idx 

In [113]:
# indice

[0,
 115231,
 230462,
 345693,
 460924,
 576155,
 691386,
 806617,
 921848,
 1037079,
 1152310,
 1267541,
 1382772,
 1498003,
 1613234,
 1728465,
 1843696,
 1958927,
 2074158,
 2189389,
 2304631]

In [114]:
# start_idx = 0
# end_idx = 2304631
# indice = list(range(start_idx,end_idx,end_idx//20))
# indice[-1] = end_idx 

# results = [None] * 20
# threads = [None] * 20
# for i in range(20):
#     threads[i] = Thread(target=data_classify,args=(labels[indice[i]:indice[i+1]],results,i))
#     threads[i].start()
# for i in range(len(threads)):
#     threads[i].join()



0it [00:00, ?it/s]

























































































0it [00:00, ?it/s]
Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
0it [00:00, ?it/s]    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_22146/242587731.py", line 10, in data_classify















0it [00:00, ?it/s]bject is not subscriptable
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_22146/242587731.py", line 10, in data_classify
















0it [00:00, ?it/s]TypeError: 'int' object is not subscriptable

0it [00:00, ?it/s]
Exception in thread Thread-20:
Tracebac

count number 뜻  
0 : 없음 , 1 : 1만 존재, 2 : 2만 존재, 3 : 1&2 존재 , 4 : 5만 존재 , 5 : 1or2 존재 하고 5 존재

In [69]:
counts

{'0': 239845, '1': 111423, '2': 34304, '3': 3523, '4': 102566, '5': 25039}

In [80]:
import numpy as np

In [84]:
print(counts['0'])

avail = np.array(list(counts.values())[1:]).sum()
print(avail)

239845
276855


In [36]:
print(count)

1410441


In [9]:
a

['../../datasets/aihub_safety/train/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/1.안전보호구만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/동탄/3.구조물_비계만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/송도호반/1.안전보호구만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/주안 재건축/2.중장비만',
 '../../datasets/aihub_safety/train/[라벨]2.공연장/부산 오페라 하우스 신축공사/5.전체',
 '../../datasets/aihub_safety/train/[라벨]8.오피스/문정동 KG 사옥 신축공사/3.구조물_비계만',
 '../../datasets/aihub_safety/train/[라벨]2.공연장/부산 오페라 하우스 신축공사/3.구조물_비계만',
 '../../datasets/aihub_safety/train/[라벨]10.주차장/고양스타필드/4.자재및공구만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/송도호반/2.중장비만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/2.중장비만',
 '../../datasets/aihub_safety/train/[라벨]3.공장/가산동 A1타워 지식산업 센터/3.구조물_비계만',
 '../../datasets/aihub_safety/train/[라벨]3.공장/가산동 A1타워 지식산업 센터/4.자재및공구만',
 '../../datasets/aihub_safety/train/[라벨]1.공동주택/송도호반/5.전체',
 '../../datasets/aihub_safety/train/[라벨]4.공항시설/인천공항 제2여객터미널 증축공사(서측)/5.전체',
 '../../datasets/aihub_s

In [17]:
len(images)

1862288

In [19]:
len(labels)

2304631

In [38]:
2224543-1862288

362255

In [44]:
2304631-1862288


442343

In [43]:
len(labels)

2304631

In [5]:
len(a)

15

In [5]:
for label in labels:
    if os.path.basename(label) == 'S2-N0509M00883.json':
        print(label)

../../datasets/aihub_safety/train/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/5.전체/S2-N0509M00883.json


In [6]:
print(len(images), len(labels))

1657489 734014


In [7]:
random.seed(42)
random.shuffle(a)

In [8]:
import json
import cv2
import numpy as np
import shutil
import random

def coord_organize(box):
    return [min(box[0],box[2]), min(box[1],box[3]), max(box[0],box[2]), max(box[1],box[3])]

def cvt_yoloformat(xyxy,w,h):
    box_w = xyxy[2] - xyxy[0]
    box_h = xyxy[3] - xyxy[1]
    return [(xyxy[0] + box_w/2)/w ,(xyxy[1] + box_h/2)/h, box_w/w, box_h/h ]


def json_transform():
    des_path = '../../datasets/aihub_safety/human_car'
    count = 0
    f_count = 0

    for i in a:
        if(os.path.basename(i)[0] not in ['5']):
            continue
        lbls = glob(i.replace('[','[[]') + '/*')
        for j in lbls:
            f_count += 1
            # print(os.path.dirname(j).split('/')[-1][0] )
            # if(os.path.dirname(j).split('/')[-1][0] == '2'):
            #     # print(j.replace('2.중장비만','5.전체'), os.path.exists(j.replace('2.중장비만','5.전체')))
            #     if os.path.exists(j.replace('2.중장비만','5.전체')):
            #         print('change',j,' @ ',j.replace('2.중장비만','5.전체'))
            #         j = j.replace('2.중장비만','5.전체')
            img_f = glob('../../datasets/aihub_safety/train/[[]원천]' + j.split('[라벨]')[1][0] + '*/' + os.path.basename(j)[:-4] +'jpg')
            try:
                img = cv2.imread(img_f[0])
            except Exception as e:
                print(e)
                print('j : ',j,'img_f : ',img_f)
            w, h = img.shape[1] , img.shape[0]

            try:
                with open(j, 'r',encoding='utf-8-sig') as f:
                    data = json.load(f)
                    annos = data['annotations']
            except Exception as e:
                print(e)
            try:
                with open(j, 'r') as f:
                    data = json.load(f)
                    annos = data['annotations']
            except Exception as e:
                print(e, '\n json data 읽기 실패')
                continue
            

            

            data_extst = False
            new_datas = []
            
            if len(annos):
                for anno in annos:
                    try:
                        cls = int(anno['class'] )
                        box = anno['box']
                    except:
                        cls = None
                        box = []
                    # print('box : ', box, 'cls : ', cls)
                    # print((len(box)) and (int(cls) < 20))
                    if (len(box)) and (int(cls) < 20):
                        data_extst = True

                        box = coord_organize(box)
                        box = cvt_yoloformat(box,w,h)
                        new_data = str(int(cls) - 1) + ' ' + ' '.join(str(k) for k in box)  + '\n'
            
                        new_datas.append(new_data)

                if(data_extst):
                    count += 1
                    try:
                        shutil.copy(img_f[0] , des_path + '/images') # image copy
                        with open(des_path + '/labels/' + os.path.basename(j)[:-4] + 'txt', 'w') as f: # write converted labeled file
                            for p in new_datas:
                                f.write(p)
                    except:
                        print('failed to write files or can\'t find image file', img_f,j)
            if f_count % 500 == 0 : print(f'f_count : {f_count}, count : {count}, folder : {i}' ) 
json_transform()

list index out of range
j :  ../../datasets/aihub_safety/train/[라벨]10.주차장/고양스타필드/5.전체/S2-N2101M24720.json img_f :  []


UnboundLocalError: local variable 'img' referenced before assignment

In [ ]:
len(labels)

734014

In [11]:
len(images)

1657489

In [12]:
## images 와 labels 폴더 포멧인 경우 random 42로 샘플링 

def sampling(img_path,label_path, des_path, size):
    if img_path[-1] == '/':
        img_path = img_path[:-1]
    if label_path[-1] == '/':
        label_path = label_path[:-1]
    if des_path[-1] == '/':
        des_path = des_path[:-1]
        
    img_source_dir = img_path
    label_source_dir = label_path
    imgs = sorted(glob(img_source_dir + '/*'))
    # print(imgs[0].replace('images','labels')[:-3] + 'txt')
    # return 0
    labels = [i.replace('images','labels')[:-3] + 'txt' for i in imgs]
    # labels = sorted(glob(label_source_dir + '/*'))
    img_des_path = des_path + '/sampled_images'
    label_des_path = des_path + '/sampled_labels'
    if not os.path.exists(img_des_path):
        os.makedirs(img_des_path)
        print(1)
    if not os.path.exists(label_des_path):
        os.makedirs(label_des_path)
    
    random.seed(42)
    random.shuffle(imgs)
    random.seed(42)
    random.shuffle(labels)

    print(imgs[:3])
    print(labels[:3])
    img_size = int(size * len(imgs))
    label_size = int(size * len(labels))
    imgs = imgs[:img_size]
    labels = labels[:label_size]

    for i in imgs:
        shutil.copy(i,img_des_path)
    for i in labels:
        shutil.copy(i,label_des_path)
    

In [13]:
ratio = 100000 / len(glob('../../datasets/aihub_safety/human_car/images/*'))


In [14]:
ratio

0.225392746861406

In [15]:
os.path.exists('../../datasets/aihub_safety/human_car/labels/S2-N0002M00001.txt')

True

In [16]:
sampling('../../datasets/aihub_safety/human_car/images','../../datasets/aihub_safety/human_car/labels','../../datasets/aihub_safety/sampled_human_car',ratio)

1
['../../datasets/aihub_safety/human_car/images/S2-N1106M07362.jpg', '../../datasets/aihub_safety/human_car/images/S2-N1105M07773.jpg', '../../datasets/aihub_safety/human_car/images/S2-N1802M07061.jpg']
['../../datasets/aihub_safety/human_car/labels/S2-N1106M07362.txt', '../../datasets/aihub_safety/human_car/labels/S2-N1105M07773.txt', '../../datasets/aihub_safety/human_car/labels/S2-N1802M07061.txt']


In [17]:
## ratio

def train_valid_test_split(img_path,label_path,des_path,ratio):
    if img_path[-1] == '/':
        img_path = img_path[:-1]
    if label_path[-1] == '/':
        label_path = label_path[:-1]
    if des_path[-1] == '/':
        des_path = des_path[:-1]

    imgs = sorted(glob(img_path + '/*'))
    random.seed(42)
    random.shuffle(imgs)
    # labels = [label_path + '/' + os.path.basename(i)[:-3] + 'txt' for i in imgs]
    labels = sorted(glob(label_path + '/*'))

    list01 = ['train', 'valid', 'test']

    for dirname in list01:
        if not os.path.exists(des_path + '/' + dirname):
            os.mkdir(des_path + '/' + dirname)
    for dirname in list01:
        for dirname2 in ['images', 'labels']:
            print(dirname2)
            if not os.path.exists(des_path + '/' + dirname + '/' + dirname2):
                os.mkdir(des_path + '/' + dirname + '/' + dirname2)

    size = len(imgs)

    

    # random.seed(42)
    # random.shuffle(labels)

    ratio_sum = 0
    for i in ratio:
        ratio_sum += i

    list02 = []
    for idx, ratio in enumerate(ratio):
        
        if len(list02):        
            list02.append(int(ratio / ratio_sum * len(imgs)) + list02[-1])
        else:
            list02.append(int(ratio / ratio_sum * len(imgs)))

        if len(list02) == 2:
            temp_imgs = imgs[list02[-2]:list02[-1]]
            temp_labels = labels[list02[-2]:list02[-1]]

        elif len(list02) == 3:
            temp_imgs = imgs[list02[-2]:]
            temp_labels = labels[list02[-2]:]
            print('test')
        else:
            temp_imgs = imgs[:list02[-1]]
            temp_labels = labels[:list02[-1]]

        for i in range(len(temp_imgs)):
            # print(temp_imgs[i])
            # print(temp_labels[i])
            
            try:
                shutil.copy(temp_labels[i],des_path + '/' + list01[idx] + '/labels')
                shutil.copy(temp_imgs[i],des_path + '/' + list01[idx] + '/images')
            except:
                pass
        
    print(list02)



In [18]:
train_valid_test_split('../../datasets/aihub_safety/sampled_human_car/sampled_images','../../datasets/aihub_safety/sampled_human_car/sampled_labels','../../datasets/aihub_safety/sampled_human_car',[17,2,1])

images
labels
images
labels
images
labels
test
[85000, 95000, 100000]
